# Importing Modules

In [1]:
import pandas as pd
import numpy as np
import sys
from functools import reduce
pd.options.display.max_columns = 200
pd.options.display.max_rows = 170

# Inspecting The Data

In [2]:
df = pd.read_csv("data/UniversityData.csv", index_col=["UnitID"])

In [3]:
institution_names_df = df[["Institution Name"]]
institution_names_df.reset_index(inplace = True)

In [4]:
df.drop(["Unnamed: 136"], axis=1, inplace=True)

In [5]:
schema = pd.read_csv("data/Schema.csv")

In [6]:
df.columns[31]

'Nonresident alien women (S2019_IS  All instructional staff)'

In [7]:
df.describe()

,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff),Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019),Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819),Core revenues total dollars (GASB) (DRVF2019),Tuition and fees as a percent of core revenues (GASB) (DRVF2019),State appropriations as percent of core revenues (GASB) (DRVF2019),Local appropriations as a percent of core revenues (GASB) (DRVF2019),Government grants and contracts as a percent of core revenues (GASB) (DRVF2019),Private gifts grants and contracts as a percent of core revenues (GASB) (DRVF2019),Investment return as a percent of core revenues (GASB) (DRVF2019),Other revenues as a percent of core revenues (GASB) (DRVF2019),Core revenues total dollars (FASB) (DRVF2019),Tuition and fees as a percent of core revenues (FASB) (DRVF2019),Government grants and contracts as a percent of core revenues (FASB) (DRVF2019),Private gifts grants contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019),Investment return as a percent of core revenues (FASB) (DRVF2019),Other revenues as a percent of core revenues (FASB) (DRVF2019),Core revenues total dollars (for-profit institutions) (DRVF2019),Tuition and fees as a percent of core revenues (for-profit institutions) (DRVF2019),Government grants and contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Private gifts grants contracts as a percent of core revenues (for-profit institutions) (DRVF2019),Invest

# Split the Dataframe by Feature Type

Categorical
<br>
Numerical
<br>
Dependent Variables

In [8]:
#list of categorical features
cat_ordinal_groups = list(schema.groupby("VariableName").groups.keys())

categorical_df = df.loc[:,cat_ordinal_groups] #dataframe with categorical features

In [9]:
df.columns[32:46] #dependent variables

Index(['Graduation rate  total cohort (DRVGR2019)',
       'Graduation rate  men (DRVGR2019)',
       'Graduation rate  women (DRVGR2019)',
       'Graduation rate  American Indian or Alaska Native (DRVGR2019)',
       'Graduation rate  Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Asian (DRVGR2019)',
       'Graduation rate  Native Hawaiian or Other Pacific Islander (DRVGR2019)',
       'Graduation rate  Black  non-Hispanic (DRVGR2019)',
       'Graduation rate  Hispanic (DRVGR2019)',
       'Graduation rate  White  non-Hispanic (DRVGR2019)',
       'Graduation rate  two or more races (DRVGR2019)',
       'Graduation rate  Race/ethnicity unknown (DRVGR2019)',
       'Graduation rate  Nonresident alien (DRVGR2019)',
       'Transfer-out rate  total cohort (DRVGR2019)'],
      dtype='object')

In [10]:
#dependent features list
dependent_var_list = list(df.columns[32:46])

dependent_df = df.loc[:,dependent_var_list]

In [11]:
#Numeric Data Frame will be all of those columns outside of dependent and categorical/ordinal

not_numeric = cat_ordinal_groups + dependent_var_list

numeric_df = df.drop(not_numeric, axis=1)

# Mapping names of categorical variables to encoded numbers

In [12]:
categorical_df.fillna("isMissing", inplace=True)

In [13]:
#convert floats to strings
for i in categorical_df.columns:
    categorical_df[i] = categorical_df[i].astype(str)

nm = categorical_df.columns

float_to_int = {str(float(i)) : str(i) for i in range(-2, 35)}

for i in nm:
    categorical_df[i].replace(float_to_int, inplace=True)

In [14]:
maps = {}
# remember from above cat_ordinal_groups is a list categorical column names
for i in cat_ordinal_groups:    
    dict_to_add = {}
   
    val_label_df = schema[schema["VariableName"] == i]

    val_label_df= val_label_df[["Value", "ValueLabel"]]


    val_label_dict = val_label_df.set_index("Value").transpose().to_dict()

    for j,v in val_label_dict.items():

        val_label_dict[j] = v['ValueLabel']
    
    maps[i] = val_label_dict


for i in cat_ordinal_groups:
    categorical_df[i].replace(maps[i], inplace=True)

In [15]:
categorical_df["City location of institution (HD2019)"] = df[df.columns[1]]

# Fixing Numeric Errors in the dataset

In [16]:
numeric_df.iloc[:,55:].describe()

,Core expenses total dollars (GASB) (DRVF2019),Instruction expenses as a percent of total core expenses (GASB) (DRVF2019),Research expenses as a percent of total core expenses (GASB) (DRVF2019),Public service expenses as a percent of total core expenses (GASB) (DRVF2019),Academic support expenses as a percent of total core expenses (GASB) (DRVF2019),Student service expenses as a percent of total core expenses (GASB) (DRVF2019),Institutional support expenses as a percent of total core expenses (GASB) (DRVF2019),Other core expenses as a percent of total core expenses (GASB) (DRVF2019),Core expenses total dollars (FASB) (DRVF2019),Instruction expenses as a percent of total core expenses (FASB) (DRVF2019),Research expenses as a percent of total core expenses (FASB) (DRVF2019),Public service expenses as a percent of total core expenses (FASB) (DRVF2019),Academic support expenses as a percent of total core expenses (FASB) (DRVF2019),Student service expenses as a percent of total core expenses (FASB) (DRVF2019),Institutional support expenses as a percent of total core expenses (FASB) (DRVF2019),Other core expenses as a percent of total core expenses (FASB) (DRVF2019),Core expenses total dollars (for-profit institutons) (DRVF2019),Instruction expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Research expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Public service expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Academic support expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Student service expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Institutional support expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Other core expenses as a percent of total core expenses (for-profit institutions) (DRVF2019),Percent of total enrollment that are American Indian or Alaska Native (DRVEF2013_RV),Percent of total enrollment that are Asian/Native Hawaiian/Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Asian (DRVEF2013_RV),Percent of total enrollment that are Native Hawaiian or Other Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Black or African American (DRVEF2013_RV),Percent of total enrollment that are Hispanic/Latino (DRVEF2013_RV),Percent of total enrollment that are White (DRVEF2013_RV),Percent of total enrollment that are Race/ethnicity unknown (DRVEF2013_RV),Percent of total enrollment that are Nonresident Alien (DRVEF2013_RV),Percent of total enrollment that are two or more races (DRVEF2013_RV),Percent of total enrollment that are women (DRVEF2013_RV),Grand total (EF2013B_RV Undergraduate Age under 25 total),Total men (EF2013B_RV Undergraduate Age under 25 total),Total women (EF2013B_RV Undergraduate Age under 25 total),Grand total (EF2013B_RV Undergraduate Age 25 and over total),Total men (EF2013B_RV Undergraduate Age 25 and over total),Total women (EF2013B_RV Undergraduate Age 25 and over total),Total price for in-district students living on campus 2012-13 (DRVIC2012_RV),Total price for in-state students living on campus 2012-13 (DRVIC2012_RV),Total price for out-of-state students living on campus 2012-13 (DRVIC2012_RV),Total price for in-district students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for out-of-state students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for in-district students living off campus (with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (with family) 2012-13 (DRVIC2012_RV),Total price for out-of-state students living off campus (with family) 2012-13 (DRVIC2012_RV),Student-to-faculty ratio (EF2019D),SAT Critical Reading 75th percentile score (IC2012_RV),SAT Math 75th percentile score (IC2012_RV),S

## no numbers should be negative, which is not the case. We have negative revenues and negative expenses

In [17]:
numeric_df.drop(numeric_df.columns[:2], axis=1, inplace=True)

In [18]:
numeric_df = numeric_df.abs()

In [19]:
numeric_df.columns[32:53]

Index(['Core revenues  total dollars (GASB) (DRVF2019)',
       'Tuition and fees as a percent of core revenues (GASB) (DRVF2019)',
       'State appropriations as percent of core revenues  (GASB) (DRVF2019)',
       'Local appropriations as a percent of core revenues (GASB) (DRVF2019)',
       'Government grants and contracts as a percent of core revenues (GASB) (DRVF2019)',
       'Private gifts  grants  and contracts as a percent of core revenues (GASB) (DRVF2019)',
       'Investment return as a percent of core revenues (GASB) (DRVF2019)',
       'Other revenues as a percent of core revenues (GASB) (DRVF2019)',
       'Core revenues  total dollars (FASB) (DRVF2019)',
       'Tuition and fees as a percent of core revenues (FASB) (DRVF2019)',
       'Government grants and contracts as a percent of core revenues (FASB) (DRVF2019)',
       'Private gifts  grants  contracts/contributions from affiliated entities as a percent of core revenues (FASB) (DRVF2019)',
       'Investment return

# Merging Financial Data

Financial data is separated between three groups: GASB, FASB, and For profit institutions. Merge into mini functions combines this data into one column instead of three separate columns for financial data.

In [20]:
def merge_into_mini(dframe, list_of_cols, final_col_name):
    df_list = []
    for i in list_of_cols:
        df_1 = pd.DataFrame(dframe.copy(deep=True)[i])
        df_1.reset_index(inplace=True)
        df_1.dropna(inplace=True)
        df_1.columns = ["UnitID", final_col_name]
        df_list.append(df_1)
    df_final = pd.concat(df_list)
    return df_final

In [21]:
numeric_df.columns[53:73]

Index(['Core expenses  total dollars (GASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Academic support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Student service expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Institutional support expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Other core expenses as a percent of total core expenses (GASB) (DRVF2019)',
       'Core expenses  total dollars (FASB) (DRVF2019)',
       'Instruction expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Research expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Public service expenses as a percent of total core expenses (FASB) (DRVF2019)',
       'Academic s

In [22]:
#create expenses classes
class Container:
    def __init__(self, col_name):
        self.main_list = []
        self.name = col_name.title().replace(" ", "_")
        for i in numeric_df.columns:
            if col_name in i:
                self.main_list.append(i)
    def __repr__(self):
        return self.name

expenses_list = [
    Container("Core expenses"), Container("Instruction expenses"), 
    Container("Research expenses"), Container("Public service expenses"),
    Container("Academic support expenses"),
    Container("Student service expenses"), Container("Institutional support expenses"),
    Container("Other core expenses")
]

revenues_list = [
    Container("Core revenues"), Container("Tuition and fees"),
    Container("Government grants"), Container("Private gifts"),
    Container("Investment return"), Container("Sales and services"),
    Container("Other revenues")   
]
                   

In [23]:
expenses_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in expenses_list]

In [24]:
expenses_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), expenses_df_list)

In [25]:
revenues_df_list = [merge_into_mini(numeric_df, i.main_list, repr(i)) for i in revenues_list]    

In [26]:
revenues_df = reduce(lambda x, y: pd.merge(x, y, on = 'UnitID', how="outer"), revenues_df_list)

In [27]:
revenues_df.fillna(value=0, inplace=True)

In [28]:
#converting to dollar amounts instead of percentages

for i in revenues_df.columns[2:]:
    col_name = i + "_As_Dollar_Amount"
    revenues_df[col_name] = revenues_df[i].apply(lambda x: x / 100) * revenues_df["Core_Revenues"]

In [29]:
revenues_df.head()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount
0,180203,7078959.0,2.0,60.0,7.0,1.0,0.0,29.0,141579.18,4247375.40,495527.13,70789.59,0.0,2052898.11
1,138558,70491879.0,17.0,17.0,2.0,0.0,0.0,28.0,11983619.43,11983619.43,1409837.58,0.00,0.0,19737726.12
2,126182,52099218.0,34.0,51.0,5.0,1.0,0.0,9.0,17713734.12,26570601.18,2604960.90,520992.18,0.0,4688929.62
3,100654,143112797.0,33.0,30.0,2.0,0.0,0.0,6.0,47227223.01,42933839.10,2862255.94,0.00,0.0,8586767.82
4,100724,134993142.0,30.0,25.0,2.0,1.0,0.0,7.0,40497942.60,33748285.50,2699862.84,1349931.42,0.0,9449519.94


In [30]:
expenses_df.fillna(value=0, inplace=True)

In [31]:
#converting to dollar amounts instead of percentages

for i in expenses_df.columns[2:]:
    col_name = i + "_As_Dollar_Amount"
    expenses_df[col_name] = expenses_df[i].apply(lambda x: x / 100) * expenses_df["Core_Expenses"]

In [32]:
finances_df = revenues_df.merge(expenses_df, how="outer", on="UnitID")

In [33]:
finances_df.describe()

,UnitID,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount
count,2814.000000,2.814000e+03,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03,2.814000e+03
mean,238196.148188,1.625889e+08,57.766525,9.251244,12.905828,5.447050,0.198294,6.656361,5.810207e+07,2.762878e+07,1.971897e+07,1.371198e+07,5.971315e+04,2.251646e+07,1.494679e+08,38.795309,2.877043,1.707534,11.036247,14.754797,23.210021,7.773987,5.760167e+07,2.232835e+07,6.404067e+06,1.705561e+07,1.305869e+07,2.001037e+07,1.333517e+07
std,117555.043407,4.990451e+08,30.155077,12.889580,17.473845,12.057034,2.456254,10.528392,1.328847e+08,1.125008e+08,1.098989e+08,1.038566e+08,2.759090e+06,1.217377e+08,4.417075e+08,12.519222,7.266436,4.163060,8.577874,9.673093,13.474402,12.024951,1.706227e+08,1.180601e+08,3.775989e+07,6.116066e+07,2.641939e+07,5.024094e+07,7.338291e+07
min,100654.000000,3.301600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,157412.500000,9.316430e+06,32.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.517664e+06,0.000000e+00,1.775469e+05,0.000000e+00,0.000000e+00,0.000000e+00,8.869112e+06,31.000000,0.000000,0.000000,6.000000,7.000000,14.000000,0.000000,2.993340e+06,0.000000e+00,0.000000e+00,7.643321e+05,9.782475e+05,2.241165e+06,0.000000e+00
50%,200306.000000,3.289078e+07,59.000000,2.000000,6.000000,2.000000,0.000000,3.000000,1.701922e+07,9.608555e+05,2.088633e+06,7.010521e+05,0.000000e+00,9.233876e+05,3.381630e+07,39.000000,0.000000,0.000000,10.000000,13.000000,21.000000,1.000000,1.275488e+07,0.000000e+00,0.000000e+00,2.936411e+06,5.377247e+06,7.126933e+06,1.367810e+05
75%,239415.500000,1.035964e+08,85.000000,16.000000,17.000000,6.000000,0.000000,8.000000,5.049367e+07,1.045950e+07,6.920550e+06,3.653437e+06,0.000000e+00,5.631486e+06,9.846859e+07,47.000000,1.000000,2.000000,13.000000,21.000000,29.000000,12.000000,4.194509e+07,8.952280e+05,9.801158e+05,1.060788e+07,1.407034e+07,1.790091e+07,5.361996e+06
max,494807.000000,8.257036e+09,367.000000,98.000000,131.000000,332.000000,85.000000,100.000000,2.051650e+09,3.137674e+09,3.302814e+09,2.754170e+09,1.463057e+08,3.214252e+09,6.894039e+09,100.000000,71.000000,53.000000,79.000000,87.000000,99.000000,100.000000,2.955601e+09,3.212136e+09,8.867553e+08,1.533947e+09,4.165172e+08,9.696544e+08,2.826556e+09


## Staff Diversity

<ol>
    <li>All expressed as percentages</li>
    <li>Split between Black, White, Hispanic/Latino, Asian, Other</li>
</ol>

In [34]:
staff_df = numeric_df.loc[:,numeric_df.columns[0:30]]

staff_df.head()

,Grand total (S2019_IS All instructional staff),Grand total men (S2019_IS All instructional staff),Grand total women (S2019_IS All instructional staff),American Indian or Alaska Native total (S2019_IS All instructional staff),American Indian or Alaska Native men (S2019_IS All instructional staff),American Indian or Alaska Native women (S2019_IS All instructional staff),Asian total (S2019_IS All instructional staff),Asian men (S2019_IS All instructional staff),Asian women (S2019_IS All instructional staff),Black or African American total (S2019_IS All instructional staff),Black or African American men (S2019_IS All instructional staff),Black or African American women (S2019_IS All instructional staff),Hispanic or Latino total (S2019_IS All instructional staff),Hispanic or Latino men (S2019_IS All instructional staff),Hispanic or Latino women (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander total (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander men (S2019_IS All instructional staff),Native Hawaiian or Other Pacific Islander women (S2019_IS All instructional staff),White total (S2019_IS All instructional staff),White men (S2019_IS All instructional staff),White women (S2019_IS All instructional staff),Two or more races total (S2019_IS All instructional staff),Two or more races men (S2019_IS All instructional staff),Two or more races women (S2019_IS All instructional staff),Race/ethnicity unknown total (S2019_IS All instructional staff),Race/ethnicity unknown men (S2019_IS All instructional staff),Race/ethnicity unknown women (S2019_IS All instructional staff),Nonresident alien total (S2019_IS All instructional staff),Nonresident alien men (S2019_IS All instructional staff),Nonresident alien women (S2019_IS All instructional staff)
UnitID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
177834,246.0,117.0,129.0,0.0,0.0,0.0,28.0,17.0,11.0,4.0,2.0,2.0,9.0,3.0,6.0,0.0,0.0,0.0,195.0,93.0,102.0,5.0,1.0,4.0,5.0,1.0,4.0,0.0,0.0,0.0
180203,13.0,7.0,6.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
222178,265.0,154.0,111.0,1.0,1.0,0.0,3.0,2.0,1.0,14.0,7.0,7.0,13.0,8.0,5.0,0.0,0.0,0.0,228.0,131.0,97.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0
138558,149.0,77.0,72.0,0.0,0.0,0.0,10.0,5.0,5.0,10.0,7.0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,124.0,62.0,62.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
488031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#make column names shorter
staff_df_col_names = []
for i in staff_df.columns:
    if "Unit" not in i: 
        col_name = i.replace(i[-36:], " instructional_staff")
    else:
        col_name = i
    
    staff_df_col_names.append(col_name)

staff_df_col_names

staff_df.columns = staff_df_col_names

staff_df.dropna(subset=[staff_df.columns[0]], inplace=True)

staff_df.reset_index(inplace=True)

## We Want to turn staff into percentages without dividing by 0

In [36]:
staff_df[staff_df[staff_df.columns[1]] == 0]
# none of these values are 0

,UnitID,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff


In [37]:
#turn staff into a percentages

for i in staff_df.columns[2:]:
    col_name = i + "_as_percentage"
    staff_df[col_name] = (staff_df[i] / staff_df[staff_df.columns[1]]) * 100

In [38]:
staff_df = staff_df.round(2)

staff_df.head()
np.where(np.isinf(staff_df))

(array([], dtype=int64), array([], dtype=int64))

In [39]:
staff_df.head()

,UnitID,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage
0,177834,246.0,117.0,129.0,0.0,0.0,0.0,28.0,17.0,11.0,4.0,2.0,2.0,9.0,3.0,6.0,0.0,0.0,0.0,195.0,93.0,102.0,5.0,1.0,4.0,5.0,1.0,4.0,0.0,0.0,0.0,47.56,52.44,0.00,0.00,0.00,11.38,6.91,4.47,1.63,0.81,0.81,3.66,1.22,2.44,0.00,0.00,0.00,79.27,37.80,41.46,2.03,0.41,1.63,2.03,0.41,1.63,0.00,0.00,0.00
1,180203,13.0,7.0,6.0,8.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.85,46.15,61.54,30.77,30.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.46,23.08,15.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,222178,265.0,154.0,111.0,1.0,1.0,0.0,3.0,2.0,1.0,14.0,7.0,7.0,13.0,8.0,5.0,0.0,0.0,0.0,228.0,131.0,97.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,58.11,41.89,0.38,0.38,0.00,1.13,0.75,0.38,5.28,2.64,2.64,4.91,3.02,1.89,0.00,0.00,0.00,86.04,49.43,36.60,1.13,0.75,0.38,0.00,0.00,0.00,1.13,1.13,0.00
3,138558,149.0,77.0,72.

## Pell Grant Recipients

In [40]:
grant_df = numeric_df.loc[:,numeric_df.columns[30:32]].reset_index()

In [41]:
grant_df.head()

,UnitID,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time undergraduates awarded federal state local or institutional grant aid (SFA1819)
0,177834,NaN,NaN
1,180203,85.0,82.0
2,222178,100.0,100.0
3,138558,91.0,86.0
4,488031,67.0,56.0


## Cohort Diversity

In [42]:
cohort_diversity_df = numeric_df.loc[:,numeric_df.columns[77:88]].reset_index()

In [43]:
cohort_diversity_df.head()

,UnitID,Percent of total enrollment that are American Indian or Alaska Native (DRVEF2013_RV),Percent of total enrollment that are Asian/Native Hawaiian/Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Asian (DRVEF2013_RV),Percent of total enrollment that are Native Hawaiian or Other Pacific Islander (DRVEF2013_RV),Percent of total enrollment that are Black or African American (DRVEF2013_RV),Percent of total enrollment that are Hispanic/Latino (DRVEF2013_RV),Percent of total enrollment that are White (DRVEF2013_RV),Percent of total enrollment that are Race/ethnicity unknown (DRVEF2013_RV),Percent of total enrollment that are Nonresident Alien (DRVEF2013_RV),Percent of total enrollment that are two or more races (DRVEF2013_RV),Percent of total enrollment that are women (DRVEF2013_RV)
0,177834,1.0,13.0,13.0,0.0,5.0,5.0,65.0,5.0,3.0,3.0,55.0
1,180203,88.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0,57.0
2,222178,0.0,1.0,1.0,0.0,8.0,11.0,69.0,2.0,4.0,3.0,58.0
3,138558,0.0,1.0,1.0,0.0,12.0,6.0,78.0,0.0,2.0,1.0,54.0
4,488031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Age and Size of cohort

In [44]:
age_df = numeric_df.loc[:,numeric_df.columns[90:96]]

age_df["Incoming_Class_Grand_Total"] = age_df.iloc[:,0] + age_df.iloc[:,1] + age_df.iloc[:,2] + age_df.iloc[:,3]

for i in age_df.columns[:-1]:
    
    col_name = i + "_as_percentage"
    age_df[col_name] = age_df[i] / age_df["Incoming_Class_Grand_Total"]

age_df = age_df.round(2)

In [45]:
age_df.head()

,Total women (EF2013B_RV Undergraduate Age under 25 total),Grand total (EF2013B_RV Undergraduate Age 25 and over total),Total men (EF2013B_RV Undergraduate Age 25 and over total),Total women (EF2013B_RV Undergraduate Age 25 and over total),Total price for in-district students living on campus 2012-13 (DRVIC2012_RV),Total price for in-state students living on campus 2012-13 (DRVIC2012_RV),Incoming_Class_Grand_Total,Total women (EF2013B_RV Undergraduate Age under 25 total)_as_percentage,Grand total (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage,Total men (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage,Total women (EF2013B_RV Undergraduate Age 25 and over total)_as_percentage,Total price for in-district students living on campus 2012-13 (DRVIC2012_RV)_as_percentage,Total price for in-state students living on campus 2012-13 (DRVIC2012_RV)_as_percentage
UnitID,,,,,,,,,,,,,
177834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180203,44.0,64.0,29.0,35.0,NaN,NaN,172.0,0.26,0.37,0.17,0.20,NaN,NaN
222178,2074.0,142.0,88.0,54.0,39900.0,39900.0,2358.0,0.88,0.06,0.04,0.02,16.92,16.92
138558,1488.0,517.0,163.0,354.0,16550.0,16550.0,2522.0,0.59,0.20,0.06,0.14,6.56,6.56
488031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Price 

In [46]:
price_df = numeric_df.loc[:,numeric_df.columns[96:103]].reset_index()

In [47]:
price_df

,UnitID,Total price for out-of-state students living on campus 2012-13 (DRVIC2012_RV),Total price for in-district students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for out-of-state students living off campus (not with family) 2012-13 (DRVIC2012_RV),Total price for in-district students living off campus (with family) 2012-13 (DRVIC2012_RV),Total price for in-state students living off campus (with family) 2012-13 (DRVIC2012_RV),Total price for out-of-state students living off campus (with family) 2012-13 (DRVIC2012_RV)
0,177834,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,180203,NaN,17030.0,17030.0,17030.0,8510.0,8510.0,8510.0
2,222178,39900.0,39900.0,39900.0,39900.0,31250.0,31250.0,31250.0
3,138558,22684.0,12619.0,12619.0,18753.0,7009.0,7009.0,13143.0
4,488031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2841,217059,28820.0,29220.0,29220.0,29220.0,19920.0,19920.0,19920.0
2842,141361,34093.0,34559.0,34559.0,34559.0,28259.0,28259.0,28259.0
2843,206695,20580.0,22694.0,22694.0,22904.0,12739.0,12739.0,12949.0
2844,204255,NaN,15610.0,15610.0,19990.0,11097.0,11097.0,15477.0


## Student Faculty Ratio 

In [48]:
student_faculty = numeric_df.loc[:,numeric_df.columns[103]].reset_index()

student_faculty.head()

,UnitID,Student-to-faculty ratio (EF2019D)
0,177834,NaN
1,180203,10.0
2,222178,15.0
3,138558,17.0
4,488031,3.0


## SAT Scores

In [49]:
test_scores = numeric_df.loc[:,numeric_df.columns[104:]].reset_index()

test_scores.drop(test_scores.columns[5:], axis = 1, inplace=True)

test_scores.drop(test_scores.columns[3], axis=1, inplace=True)

In [50]:
for i in test_scores.columns:
    if "Unit" not in i:
        col_name = i + "_isNaN"
        test_scores[col_name] = test_scores[i].isna() * 1

In [51]:
test_scores.head()

,UnitID,SAT Critical Reading 75th percentile score (IC2012_RV),SAT Math 75th percentile score (IC2012_RV),ACT Composite 75th percentile score (IC2012_RV),SAT Critical Reading 75th percentile score (IC2012_RV)_isNaN,SAT Math 75th percentile score (IC2012_RV)_isNaN,ACT Composite 75th percentile score (IC2012_RV)_isNaN
0,177834,NaN,NaN,NaN,1,1,1
1,180203,NaN,NaN,NaN,1,1,1
2,222178,600.0,600.0,27.0,0,0,0
3,138558,420.0,410.0,17.0,0,0,0
4,488031,NaN,NaN,NaN,1,1,1


# Merging all cleaned dataframes on UnitId

In [52]:
cleaned_df_list = [ 
    institution_names_df, finances_df, staff_df, grant_df, 
    cohort_diversity_df, age_df, price_df, student_faculty, 
    test_scores, categorical_df, dependent_df
]

cleaned_df = reduce(lambda x, y: pd.merge(x, y, on='UnitID', how="outer"), cleaned_df_list)

In [53]:
cleaned_df.shape

(2846, 152)

In [54]:
#drop where total graduation rate is missing
cleaned_df.dropna(subset=[dependent_df.columns[0]], inplace=True)

In [55]:
cleaned_df.head()

,UnitID,Institution Name,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of 

In [56]:
cleaned_df.to_csv("data/cleaned_df.csv")